# Bedrock with LangChain - Explain/Interpret a code snippet or program 
> *This notebook should work well with the **`Data Science 3.0`** kernel in SageMaker Studio*

## Introduction

In this notebook we show you how to explain or interpret a given code snippet or program.

[LangChain](https://python.langchain.com/docs/get_started/introduction.html) is a framework for developing applications powered by language models. The key aspects of this framework allow us to augment the Large Language Models by chaining together various components to create advanced use cases.

In this notebook we will use the Bedrock API provided by LangChain. The prompt used in this example creates a custom LangChain prompt template for adding context to the code explain request. 

**Note:** *This notebook can be run within or outside of AWS environment.*

#### Context
In this notebook we will leverage the LangChain framework and explore Bedrock API with the help of `PromptTemplates`. `PrompTemplates` allow you to create generic shells which can be populated with information later and get model outputs based on different scenarios.

As part of this notebook we will explore the use of Amazon Bedrock integration within LangChain framework and how it could be used to generate or explain code with the help of `PromptTemplate`.

#### Pattern
We will simply provide the LangChain implementation of Amazon Bedrock API with an input consisting of a task, an instruction and an input for the model under the hood to generate an output without providing any additional example. The purpose here is to demonstrate how the powerful LLMs easily understand the task at hand and generate compelling outputs.

![](./images/code-interpret-langchain.png)

#### Use case
To demonstrate the code generation capability of models in Amazon Bedrock, let's take the use case of code explain.

#### Persona
You are Joe, a Java software developer, has been tasked to support a legacy C++ application for Vehicle Fleet Management. You need help to explain or interpret certain complex C++ code snippets as you are performing analyis to identify the business logic and potential problems with the code.

#### Implementation
To fulfill this use case, we will show you how you can Amazon Bedrock API with LangChain to explain C++ code snippets.


In [1]:
from IPython.core.display import HTML
from IPython.display import display_markdown, Markdown
import boto3

HTML("<script>Jupyter.notebook.kernel.restart()</script>")
boto3_bedrock = boto3.client('bedrock-runtime')

## Invoke the Bedrock LLM Model

We'll begin with creating an instance of Bedrock class from llms. This expects a `model_id` which is the ARN of the model available in Amazon Bedrock. 

Optionally you can pass on a previously created boto3 client as well as some `model_kwargs` which can hold parameters such as `temperature`, `topP`, `maxTokenCount` or `stopSequences` (more on parameters can be explored in Amazon Bedrock console).

Check [documentation](https://docs.aws.amazon.com/bedrock/latest/userguide/model-ids-arns.html) for Available text generation model Ids under Amazon Bedrock.

Note that different models support different `model_kwargs`.

In [2]:
from langchain_aws import ChatBedrockConverse


textgen_llm = ChatBedrockConverse(
    model_id="amazon.nova-lite-v1:0",
    client=boto3_bedrock,
    max_tokens=None,
    temperature=0.5
)

## Create a LangChain custom prompt template

By creating a template for the prompt we can pass it different input variables to it on every run. This is useful when you have to generate content with different input variables that you may be fetching from a database.

In [3]:
# Vehicle Fleet Management Code written in C++
sample_code = """
#include <iostream>
#include <string>
#include <vector>

class Vehicle {
protected:
    std::string registrationNumber;
    int milesTraveled;
    int lastMaintenanceMile;

public:
    Vehicle(std::string regNum) : registrationNumber(regNum), milesTraveled(0), lastMaintenanceMile(0) {}

    virtual void addMiles(int miles) {
        milesTraveled += miles;
    }

    virtual void performMaintenance() {
        lastMaintenanceMile = milesTraveled;
        std::cout << "Maintenance performed for vehicle: " << registrationNumber << std::endl;
    }

    virtual void checkMaintenanceDue() {
        if ((milesTraveled - lastMaintenanceMile) > 10000) {
            std::cout << "Vehicle: " << registrationNumber << " needs maintenance!" << std::endl;
        } else {
            std::cout << "No maintenance required for vehicle: " << registrationNumber << std::endl;
        }
    }

    virtual void displayDetails() = 0;

    ~Vehicle() {
        std::cout << "Destructor for Vehicle" << std::endl;
    }
};

class Truck : public Vehicle {
    int capacityInTons;

public:
    Truck(std::string regNum, int capacity) : Vehicle(regNum), capacityInTons(capacity) {}

    void displayDetails() override {
        std::cout << "Truck with Registration Number: " << registrationNumber << ", Capacity: " << capacityInTons << " tons." << std::endl;
    }
};

class Car : public Vehicle {
    std::string model;

public:
    Car(std::string regNum, std::string carModel) : Vehicle(regNum), model(carModel) {}

    void displayDetails() override {
        std::cout << "Car with Registration Number: " << registrationNumber << ", Model: " << model << "." << std::endl;
    }
};

int main() {
    std::vector<Vehicle*> fleet;

    fleet.push_back(new Truck("XYZ1234", 20));
    fleet.push_back(new Car("ABC9876", "Sedan"));

    for (auto vehicle : fleet) {
        vehicle->displayDetails();
        vehicle->addMiles(10500);
        vehicle->checkMaintenanceDue();
        vehicle->performMaintenance();
        vehicle->checkMaintenanceDue();
    }

    for (auto vehicle : fleet) {
        delete vehicle; 
    }

    return 0;
}
"""

In [4]:
from langchain.prompts import PromptTemplate

# Create a prompt template that has multiple input variables
multi_var_prompt = PromptTemplate(
    input_variables=["code", "programmingLanguage"], 
    template="""

Human: You will be acting as an expert software developer in {programmingLanguage}. 
You will explain the below code and highlight if there are any red flags or where best practices are not being followed.
<code>
{code}
</code>

Assistant:"""
)

# Pass in values to the input variables
prompt = multi_var_prompt.format(code=sample_code, programmingLanguage="C++")


### Explain C++ Code for Vehicle Fleet management using Amazon Bedrock and LangChain

In [5]:
response = textgen_llm.invoke(prompt)

code_explanation = response.content

display_markdown(Markdown(code_explanation))

Certainly! Let's go through the code step by step and highlight any potential issues or areas where best practices could be improved.

### Code Overview

The code defines a hierarchy of classes for different types of vehicles. It includes a base class `Vehicle` and derived classes `Truck` and `Car`. Each class has methods to add miles, check for maintenance, perform maintenance, and display details.

### Detailed Explanation

#### Vehicle Class

- **Protected Members**: 
  - `registrationNumber`: Stores the vehicle's registration number.
  - `milesTraveled`: Tracks the total miles the vehicle has traveled.
  - `lastMaintenanceMile`: Tracks the miles at the last maintenance.

- **Constructor**: Initializes the registration number, sets miles traveled and last maintenance mile to 0.

- **Virtual Methods**:
  - `addMiles`: Adds miles to the vehicle.
  - `performMaintenance`: Sets the last maintenance mile to the current miles and prints a message.
  - `checkMaintenanceDue`: Checks if the vehicle needs maintenance based on the miles traveled since the last maintenance.
  - `displayDetails`: Pure virtual function to be implemented by derived classes.
  - Destructor: Prints a message indicating the destructor is called.

#### Truck and Car Classes

- Both derived classes override the `displayDetails` method to provide specific details about the vehicle.

### Main Function

- A vector of `Vehicle*` is created to store pointers to different types of vehicles.
- A `Truck` and a `Car` are created and added to the fleet.
- The code iterates over the fleet, calling various methods on each vehicle.
- After usage, the dynamically allocated vehicles are deleted to prevent memory leaks.

### Red Flags and Best Practices

1. **Memory Management**:
   - **Red Flag**: The code manually deletes the dynamically allocated `Vehicle` objects. This is error-prone and can lead to memory leaks if not done correctly.
   - **Best Practice**: Use smart pointers (`std::unique_ptr` or `std::shared_ptr`) to handle memory automatically.

2. **Virtual Destructor**:
   - **Red Flag**: The destructor in the `Vehicle` class is not marked as virtual.
   - **Best Practice**: Mark the destructor as virtual to ensure proper cleanup of derived class instances when deleted through a base class pointer.

3. **Pure Virtual Function**:
   - **Best Practice**: The `displayDetails` function is correctly marked as pure virtual in the base class, ensuring derived classes implement it.

4. **Code Duplication**:
   - **Red Flag**: The `checkMaintenanceDue` and `performMaintenance` methods print messages directly. This can be improved by encapsulating the output logic.
   - **Best Practice**: Consider using a separate logging mechanism or at least encapsulate the output in a way that it can be easily modified.

5. **Magic Numbers**:
   - **Red Flag**: The number 10000 in the `checkMaintenanceDue` method is a magic number.
   - **Best Practice**: Define a constant for such values to improve readability and maintainability.

### Improved Code

Here's an improved version of the code incorporating the best practices mentioned:

```cpp
#include <iostream>
#include <string>
#include <vector>
#include <memory>

class Vehicle {
protected:
    std::string registrationNumber;
    int milesTraveled;
    int lastMaintenanceMile;

public:
    Vehicle(std::string regNum) : registrationNumber(regNum), milesTraveled(0), lastMaintenanceMile(0) {}

    virtual void addMiles(int miles) {
        milesTraveled += miles;
    }

    virtual void performMaintenance() {
        lastMaintenanceMile = milesTraveled;
    }

    virtual void checkMaintenanceDue() const {
        if ((milesTraveled - lastMaintenanceMile) > MAINTENANCE_INTERVAL) {
            std::cout << "Vehicle: " << registrationNumber << " needs maintenance!" << std::endl;
        } else {
            std::cout << "No maintenance required for vehicle: " << registrationNumber << std::endl;
        }
    }

    virtual void displayDetails() const = 0;

    virtual ~Vehicle() {
        std::cout << "Destructor for Vehicle" << std::endl;
    }
};

class Truck : public Vehicle {
    int capacityInTons;

public:
    Truck(std::string regNum, int capacity) : Vehicle(regNum), capacityInTons(capacity) {}

    void displayDetails() const override {
        std::cout << "Truck with Registration Number: " << registrationNumber << ", Capacity: " << capacityInTons << " tons." << std::endl;
    }
};

class Car : public Vehicle {
    std::string model;

public:
    Car(std::string regNum, std::string carModel) : Vehicle(regNum), model(carModel) {}

    void displayDetails() const override {
        std::cout << "Car with Registration Number: " << registrationNumber << ", Model: " << model << "." << std::endl;
    }
};

const int MAINTENANCE_INTERVAL = 10000;

int main() {
    std::vector<std::unique_ptr<Vehicle>> fleet;

    fleet.push_back(std::make_unique<Truck>("XYZ1234", 20));
    fleet.push_back(std::make_unique<Car>("ABC9876", "Sedan"));

    for (auto& vehicle : fleet) {
        vehicle->displayDetails();
        vehicle->addMiles(10500);
        vehicle->checkMaintenanceDue();
        vehicle->performMaintenance();
        vehicle->checkMaintenanceDue();
    }

    return 0;
}
```

### Summary

- **Smart Pointers**: Use `std::unique_ptr` for automatic memory management.
- **Virtual Destructor**: Ensure the destructor in the base class is virtual.
- **Magic Numbers**: Define constants for magic numbers.
- **Encapsulation**: Consider encapsulating output logic to improve flexibility.

## Summary

To conclude we learnt that invoking the LLM without any context might not yield the desired results. By adding context and further using the prompt template to constrain the output from the LLM we are able to successfully get our desired output